# Settings & Options

PowerWorld has many simulation settings - solver iteration limits,
convergence tolerances, control flags, GIC parameters, and more.
Traditionally these require verbose getter/setter calls through
the SimAuto API. ESA++ exposes the most commonly used settings as
**Python attributes** on `PowerWorld` and `pw.gic`, so reading or
changing a setting is as simple as reading or assigning a variable.

```python
from esapp import PowerWorld
from esapp.components import *

pw = PowerWorld("path/to/case.pwb")
```

In [1]:
from esapp import PowerWorld
from esapp.components import *
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

'open' took: 7.1384 sec


## Solver Options

Solver options control how the Newton-Raphson power flow behaves.
They live directly on the `PowerWorld` object as Python attributes —
just read or assign them like normal variables. Boolean options
use `True`/`False`; numeric options use `int` or `float`.
Changes take effect immediately in PowerWorld.

In [ ]:
# Read current settings
pw.max_iterations, pw.convergence_tol, pw.flat_start, pw.dc_mode

(50, 1e-07, False, False)

In [ ]:
# Change numeric settings
pw.max_iterations = 200
pw.convergence_tol = 1e-5

# Toggle boolean settings
pw.dc_mode = True
assert pw.dc_mode is True
pw.dc_mode = False

pw.max_iterations, pw.convergence_tol, pw.dc_mode

(200, 1e-05, False)

### Solver option reference

**Iteration & convergence:**

| Attribute | Type | Description |
|---|---|---|
| `max_iterations` | int | Maximum Newton-Raphson iterations |
| `max_vcl_iterations` | int | Maximum voltage-constrained loop iterations |
| `convergence_tol` | float | Power flow convergence tolerance |
| `min_volt_i_load` | float | Minimum voltage for constant-current loads (pu) |
| `min_volt_s_load` | float | Minimum voltage for constant-impedance loads (pu) |

**Boolean controls:**

| Attribute | Description |
|---|---|
| `do_one_iteration` | Solve only one Newton iteration per call |
| `disable_opt_mult` | Disable optimal multiplier acceleration |
| `flat_start` | Start from flat voltage profile (1.0 pu, 0 deg) |
| `dc_mode` | Enable DC power flow approximation |
| `inner_ss_check` | Check switched shunt controls in inner loop |
| `disable_gen_mvr_check` | Disable generator MVR limit checking |
| `inner_check_gen_vars` | Check generator VAR limits in inner loop |
| `inner_backoff_gen_vars` | Back off generator VAR limits in inner loop |
| `check_taps` | Check transformer tap adjustments |
| `check_shunts` | Check switched shunt adjustments |
| `check_phase_shifters` | Check phase shifter adjustments |
| `prevent_oscillations` | Prevent control oscillations |
| `disable_angle_rotation` | Disable automatic angle rotation to slack |
| `allow_mult_islands` | Allow multiple island solutions |
| `eval_solution_island` | Evaluate solution quality per island |
| `enforce_gen_mw_limits` | Enforce generator MW output limits |

## GIC Options

GIC options are accessed through `pw.gic` and work the same way —
read by accessing the attribute, write by assigning to it.

In [ ]:
# Read GIC settings
pw.gic.pf_include, pw.gic.ts_include, pw.gic.calc_mode

(False, False, 'SnapShot')

In [ ]:
# Change GIC settings
pw.gic.pf_include = True
pw.gic.efield_angle = 90.0
pw.gic.efield_mag = 1.0

pw.gic.pf_include, pw.gic.efield_angle, pw.gic.efield_mag

(True, '90', '1')

`configure()` sets multiple GIC options at once with sensible
defaults. With no arguments it enables GIC in power flow with
snapshot mode.

In [ ]:
pw.gic.configure()  # defaults: pf_include=True, ts_include=False, calc_mode="SnapShot"
pw.gic.pf_include, pw.gic.ts_include, pw.gic.calc_mode

(True, False, 'SnapShot')

`settings()` returns every GIC option as a DataFrame, including
options without a dedicated attribute.

In [ ]:
pw.gic.settings().head(10)

,VariableName,ValueField
0,AutoXFMaxTurnsRatio,4
1,BusNoSub,None (Ungrounded)
2,CalcInducedDCVoltEquiv,NO
3,CalcInducedDCVoltLength,1
4,CalcInducedDCVoltLowR,NO
5,CalcMaxDirection,YES
6,CalcMode,SnapShot
7,DistXFConfigDefault,GWye
8,EField3dFileMerge,YES
9,EField3dFileMultLoad,0


### Available GIC options

**Core:**

| Attribute | Type | Description |
|---|---|---|
| `pf_include` | bool | Include GIC effects in power flow |
| `ts_include` | bool | Include GIC effects in transient stability |
| `calc_mode` | str | Calculation mode (SnapShot, TimeVarying, etc.) |

**Electric field:**

| Attribute | Type | Description |
|---|---|---|
| `efield_angle` | float | Storm direction in degrees |
| `efield_mag` | float | Field magnitude in V/distance |
| `calc_max_direction` | bool | Auto-calculate maximum E-field direction |

**Calculation controls:**

| Attribute | Type | Description |
|---|---|---|
| `update_line_volts` | bool | Auto-update line DC voltages |
| `skip_equiv_lines` | bool | Skip DC voltage on equivalent lines |
| `skip_low_r_lines` | bool | Skip DC voltage on low-R lines |
| `min_kv` | float | Minimum nominal kV to include GIC effects |
| `segment_length_km` | float | Max line segment length (km) |
| `bus_no_sub` | str | Auto-insert option for buses without substations |
| `hotspot_include` | bool | Include hotspot calculations |

## Under the Hood

These attributes are Python [descriptors](https://docs.python.org/3/howto/descriptor.html)
defined in `esapp._descriptors`. Each one maps a Python name to a
PowerWorld option field. Boolean options stored as `"YES"`/`"NO"`
in PowerWorld are automatically converted to/from `True`/`False`.
You can inspect the mapping at the class level:

In [ ]:
desc = type(pw).max_iterations
desc.key, desc.is_bool

('MaxItr', False)